Initially loading the Dataset and pre-req operations

In [ ]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from transformers import pipeline
nltk.download('punkt_tab')
# Load dataset
file_path = "/content/Cleaned Smart Email Dataset.csv"
df = pd.read_csv(file_path)